# 술 추천 알고리즘

도수 유사도는 `euclidean distance`로 구했고, 풍미 및 기타 정보의 유사도는 `cosine similarity`로 구했다.  
술의 고유 id를 받아 비슷한 술의 고유 id를 return한다.

### general_recommendation():
도수 유사도 50% 풍미(4가지) 유사도 50% 고려해 주종 상관 없이 비슷한 술을 찾아줌

### wine_recommendation():
도수 유사도 50% 기타(category, origin, producer, wine_grape, flavour) 50% 고려해 같은 대분류 내에서 비슷한 와인을 찾아줌

### the used variables of each alcohols

- 와인: 주종, 중분류, 주류명, 생산국가, 생산자/제조사, 주요품종, 도수, 음용온도, 추천음식, 맛
- 전통주 : 주종, 주류명, 생산국가, 생산자/제조사, 주원료, 용량, 도수  
- 브랜디 : '주종','주류명','생산국가','용량','도수','특징'
- 위스키 : '주종','주류명','생산국가','용량','도수','특징'
- 국내주류 : '주종','주류명','생산국가','생산자/제조사','용량','도수','특징'

- 막걸리는 도수, 주원료, 제조사 고려해 와인 추천 

In [2]:
import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler
from string import punctuation
import re


In [32]:
mark = pd.read_csv('makgeolli_fin.csv')
mark = mark.drop(['Unnamed: 0'],axis=1)
mark

,주종,상품명,생산국가,제조사,주원료,용량,도수
0,전통주,입장탁주,대한민국,(농) 입장주조㈜,국내산 쌀,750ml,7.0
1,전통주,톡 한잔 소주,대한민국,(농)(유)대마주조,"국내산 보리, 쌀",500ml,30.0
2,전통주,자연담은 복분자막걸리,대한민국,(농)국순당고창명주㈜,"국내산 복분자(고창), 정제수",360ml,6.0
3,전통주,로얄 안동소주,대한민국,(농)유토피아㈜,국내산 쌀,"600ml, 800ml",45.0
4,전통주,지란지교,대한민국,(유)친구들의 술 지란지교,"찹쌀,멥쌀,전통누룩(밀),정제수",500ml,17.0
...,...,...,...,...,...,...,...
846,전통주,화백,대한민국,화백,"멥쌀(국내산), 찹쌀(국내산), 누룩(국내산 밀함유), 효모, 정제수",500ml,25.0
847,전통주,산천어막걸리,대한민국,화천주가,"정제수,팽화미(쌀/외국산),입국(쌀),국(밀),설탕,정제효소제, 효모,아스파탐(감미...",750ml,6.0
848,전통주,황금보리 증류주 17,대한민국,황금보리(유)농업회사법인,"보리(국내산) 39.9%, 누룩(국내산)3.3%, 효소(액화효소), 효모(향미증진제...",375ml,17.0
849,전통주,황금보리 증류주 25,대한민국,황금보리(유)농업회사법인,"정제수, 보리(국내산) 40.05%, 누룩, 효소(액화효소), 효모, 과당, 구연산...",375ml,25.0


In [33]:
# 도수 유사도 구하기
percent = mark['도수']
dist_pair = []

# y축에 임의로 0을 부여한 거리 순서쌍 생성
for i in range(0,len(percent)):
    temp = []
    temp.append(mark.loc[i]['도수'])
    temp.append(0)
    dist_pair.append(temp)

# get a distance matrix
df = pd.DataFrame(dist_pair, columns=['x', 'y'])
dist_matrix = distance_matrix(df.values, df.values)

# 정규화
min_max_scaler = MinMaxScaler()
regularised = min_max_scaler.fit_transform(dist_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
one_matrix = np.ones((851,851))

final_dist = one_matrix - regularised

# 확인
print(final_dist)

[[1.         0.25806452 0.98181818 ... 0.77272727 0.5        0.10810811]
 [0.57407407 1.         0.56363636 ... 0.70454545 0.86111111 0.72972973]
 [0.98148148 0.22580645 1.         ... 0.75       0.47222222 0.08108108]
 ...
 [0.81481481 0.58064516 0.8        ... 1.         0.77777778 0.37837838]
 [0.66666667 0.83870968 0.65454545 ... 0.81818182 1.         0.59459459]
 [0.38888889 0.67741935 0.38181818 ... 0.47727273 0.58333333 1.        ]]


In [34]:
mark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   주종      851 non-null    object 
 1   상품명     851 non-null    object 
 2   생산국가    851 non-null    object 
 3   제조사     851 non-null    object 
 4   주원료     851 non-null    object 
 5   용량      851 non-null    object 
 6   도수      851 non-null    float64
dtypes: float64(1), object(6)
memory usage: 46.7+ KB


In [35]:
# 전통주 word list 만들기
mark_list = [] # empty list

# 전통주 index list 추출
mark_idx = mark.index[mark['주종'] == "전통주"].tolist()

# 도수, 주원료 

for i in mark_idx:
    temp = ""
    temp = temp + mark.loc[i]['주원료'] #.replace(" ", "")
    
#     for flavour in data.loc[i]['flavour']:
#         temp = temp + flavour

    mark_list.append(temp)

# 전통주 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
mark_matrix = count.fit_transform(mark_list)

# generating the cosine similarity matrix
mark_sim = cosine_similarity(mark_matrix, mark_matrix)

# 전통주 도수 유사도 구하기
mark_pair = []

# y축을 임의로 0을 부여한 거리 순서쌍 생성
for i in mark_idx:
    temp = []
    temp.append(mark.loc[i]['도수'])
    temp.append(0)
    mark_pair.append(temp)

# get a distance matrix
mark_df = pd.DataFrame(mark_pair, columns=['x', 'y'])
mark_matrix = distance_matrix(mark_df.values, mark_df.values)

# 정규화
min_max_scaler = MinMaxScaler()
mark_regularised = min_max_scaler.fit_transform(mark_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
mark_one_matrix = np.ones((len(mark_idx),len(mark_idx)))

mark_final_dist = mark_one_matrix - mark_regularised

# 도수 유사도, 타 정보 유사도 각각 0.5씩 weight 부여 후 새로운 matrix 생성
mark_new_sim = 0.5 * mark_sim + 0.5 * mark_final_dist

# 확인
print(mark_new_sim)

[[1.         0.48258565 0.74090909 ... 0.64456253 0.39433757 0.1927291 ]
 [0.64059043 1.         0.45859488 ... 0.62613401 0.6346797  0.560981  ]
 [0.74074074 0.28967992 1.         ... 0.56864917 0.38044868 0.17921559]
 ...
 [0.6656063  0.56418386 0.59364917 ... 1.         0.79883468 0.61885811]
 [0.4776709  0.62347898 0.47161029 ... 0.8190367  1.         0.73764972]
 [0.33311949 0.53482581 0.32958414 ... 0.66830529 0.73201909 1.        ]]


In [ ]:
# 해당 id의 술이 전통주인지 체크
def is_mark(input_id):
    temp_idx = mark.index[mark['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    result = mark.loc[temp_idx[0]]['주종'] == "전통주"
    return result

#  전통주의 고유 id를 넣으면 해당 전통주와 비슷한 Top 10 전통주의 주류명를 return
def mark_recommendation(input_id, mark_new_sim = mark_new_sim):
    
    # 고유 id로 index 찾기 
    idx = mark.index[mark['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # wine_idx list 내에서 몇번째 와인인지 구하기
    w_idx = wine_idx.index(idx[0])
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(wine_new_sim[w_idx]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    wine_top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if wine_top_10_indexes[0] == w_idx:
        wine_top_10_indexes = list(score_series.iloc[1:12].index)
        wine_top_10_indexes.remove(w_idx)
    
    # 고유 id를 담기 위한 empty list 생성
    wine_top_10_id = []
    
    # id list
    for i in wine_top_10_indexes:
        index = wine_idx[i] # wine list에서 몇번째인지가 아니라 전체 술 list에서 몇번째인지 구함
        id = wine.loc[index]['상품명']
        wine_top_10_id.append(id)
    
    return wine_top_10_id